In [16]:
# !pip install ultralytics opencv-python


In [17]:
import torch
print(torch.cuda.is_available())


True


| เรื่อง           | YOLOv8        | YOLO11 / ใหม่กว่า |
| ---------------- | ------------- | ----------------- |
| ByteTrack        | ✅ tested      | ⚠️ ต้องลอง        |
| MMpose           | ✅ ใช้จริงเยอะ | ❓                 |
| Export TensorRT  | ✅ ง่าย        | ⚠️                |
| Community bugfix | เยอะ          | น้อย              |
| Reproducibility  | สูง           | ต่ำกว่า           |


In [18]:
from ultralytics import YOLO

# โหลด pretrained COCO
model = YOLO("yolov8l.pt")


In [19]:
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO
from collections import defaultdict
from IPython.display import clear_output

In [20]:
model = YOLO("yolov8l.pt")
cap = cv2.VideoCapture("muay_thai_clip.mp4")

frame_id = 0
rows = []   # เก็บข้อมูลก่อนแปลงเป็น DataFrame

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(
        frame,
        conf=0.25,
        iou=0.7,
        classes=[0],
        tracker="bytetrack.yaml",
        persist=True,
        device=0,
        verbose=False
    )

    r = results[0]

    if r.boxes is not None and r.boxes.id is not None:
        boxes = r.boxes.xyxy.cpu().numpy()
        ids = r.boxes.id.cpu().numpy().astype(int)

        for box, track_id in zip(boxes, ids):
            x1, y1, x2, y2 = map(int, box)
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2

            # เก็บทุกเฟรม ทุก ID
            rows.append({
                "frame": frame_id,
                "track_id": track_id,
                "x1": x1,
                "y1": y1,
                "x2": x2,
                "y2": y2,
                "cx": cx,
                "cy": cy
            })

            # display
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(
                frame,
                f"ID {track_id}",
                (x1, y1 - 5),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0, 255, 0),
                2
            )

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 6))
    plt.imshow(frame_rgb)
    plt.axis("off")
    plt.title(f"Frame {frame_id}")
    plt.show()

    clear_output(wait=True)
    frame_id += 1

cap.release()

# สร้าง DataFrame
df = pd.DataFrame(rows)

print("========== SUMMARY ==========")
print(f"Total frames processed : {frame_id}")
print(f"Unique IDs             : {df['track_id'].nunique()}")


========== SUMMARY ==========
Total frames processed : 3866
Unique IDs             : 63


In [21]:
df.to_csv("track_data.csv", index=False)

In [22]:
df.head()

,frame,track_id,x1,y1,x2,y2,cx,cy
0,0,1,1080,501,1174,807,1127,654
1,0,2,1504,588,1650,977,1577,782
2,0,3,733,512,804,797,768,654
3,0,4,879,536,986,881,932,708
4,0,5,657,582,746,974,701,778


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35135 entries, 0 to 35134
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   frame     35135 non-null  int64
 1   track_id  35135 non-null  int64
 2   x1        35135 non-null  int64
 3   y1        35135 non-null  int64
 4   x2        35135 non-null  int64
 5   y2        35135 non-null  int64
 6   cx        35135 non-null  int64
 7   cy        35135 non-null  int64
dtypes: int64(8)
memory usage: 2.1 MB
